In [1]:
import json, datetime, math
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import scale
import operator
from sklearn.metrics import calinski_harabaz_score
from sklearn.cluster import KMeans
import wordcloud

In [2]:
with open("data/orig_en_train.json") as f:
    data = json.load(f)
    # print("number of tweets", len(data))

In [4]:
all_hs = defaultdict(int)
hs_likes = defaultdict(int)

for d in data:
    curr_hashtags = d['hashtags']
    curr_hashtags = curr_hashtags[1:-1].split(', ')
    curr_hashtags = [x for x in curr_hashtags if x != ""]
    for hs in curr_hashtags:
        if hs!="":
            all_hs[hs] += 1
    if int(d['like_count']) > 0:
        for hs in curr_hashtags:
            if hs!="":
                hs_likes[hs] += 1

sorted_hs = sorted(all_hs.items(), key=operator.itemgetter(1))
sorted_hsl = sorted(hs_likes.items(), key=operator.itemgetter(1))
    
quality_hs = []
quality_hsl = []
for hs in sorted_hs:
    if hs[1]>7:
        quality_hs.append(hs)
for hs in sorted_hsl:
    if hs[1]>7:
        quality_hsl.append(hs)

hs_index = defaultdict(int)
ind_hs = defaultdict(str)
for ind, hs in enumerate(quality_hs):
    hs_index[hs[0]] = ind
    ind_hs[ind] = hs[0]
    
all_user = set()
for d in data:
    all_user.add(d['userid'])
users = list(all_user)
user_ind = {}
ind_user = {}
user_hs = []
for ind, u in enumerate(users):
    user_hs.append([0]*len(quality_hs))
    user_ind[u] = ind
    ind_user[ind] = u
    
for d in data:
    curr_hashtags = d['hashtags']
    curr_hashtags = curr_hashtags[1:-1].split(', ')
    curr_hashtags = [x for x in curr_hashtags if x != ""]
    for hs in curr_hashtags:
        user_hs[user_ind[d['userid']]][hs_index[hs]] += 1

In [5]:
filtered_user_hs = []
for d in user_hs:
    if d != [0]*len(quality_hs):
        filtered_user_hs.append(d)
filtered_user_hs = np.array(filtered_user_hs, dtype=np.float32)

In [6]:
user_hs_sums = []
for d in filtered_user_hs:
    user_hs_sums.append(sum(d))

In [7]:
for i in range(filtered_user_hs.shape[0]):
    filtered_user_hs[i] = np.divide(filtered_user_hs[i], user_hs_sums[i])

In [8]:
scaled_user_hs = scale(filtered_user_hs)

/home/santi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/santi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [9]:
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(scaled_user_hs)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    print(i, calinski_harabaz_score(scaled_user_hs, labels))

2 26.15858515390863
3 19.136288237237693
4 19.36554009525447
5 10.529072244753463
6 14.879765551453158
7 12.57472559291928
8 12.285952328690142
9 12.725429928950879


In [21]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(scaled_user_hs)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

sorted_center_hs = []
for d in centers:
    sorted_center_hs.append(np.argsort(d))

In [25]:
for d in sorted_center_hs[0][-20:]:
    print(ind_hs[d])

politics
WELOVEYOUTOONIALL
ccot
phosphorusdisaster
WakeUpAmerica
TrumpForPresident
iTunes
StopTheGOP
quotes
PJNET
GOPDebate
DeadHorse
KochFarms
IslamKills
tcot
BlackLivesMatter
Chernobyl2015
ColumbianChemicals
FukushimaAgain
ThingsMoreTrustedThanHillary
